In [3]:
#imports
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import sys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
#Driver for firefox and loading the Eweb url
driver = webdriver.Firefox()
driver.implicitly_wait(10)

driver.get("https://www.easternct.edu/faculty-directory/index.html")

In [4]:
#selecting SearchButton
Search = driver.find_element(by = 'id', value = 'directorySearchButton')
Search.click()

In [ ]:
#Locating the HTML where the info is stored
table = driver.find_element(by = 'tag name', value = "az-content")

In [ ]:
#The loop + the storage array
OH = []

for professors in table[1:]:
    rows = driver.find_element(by = 'class')

    for officeHours in rows:
        indicies = [0,2]
        data = []
        for i in officeHours[indicies]:
            x = driver.find_

In [ ]:
print(len(table))